In [1]:
%pip install --upgrade langchain langchain-community langchain-openai langchain-experimental neo4j python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 KB 2.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI, AzureChatOpenAI
import os

llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")

# llm = AzureChatOpenAI(
#                 deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), 
#                 model_name=os.getenv("AZURE_OPENAI_MODEL_NAME"), 
#                 temperature=0
#             )

llm_transformer = LLMGraphTransformer(llm=llm)

In [4]:
from langchain_core.documents import Document

text = """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Pierre Curie', type='Person'), type='MARRIED'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='University Of Paris', type='Organization'), type='PROFESSOR')]


In [6]:
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person'), Node(id='Pierre Curie', type='Person'), Node(id='University Of Paris', type='Organization')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Pierre Curie', type='Person'), type='SPOUSE'), Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='University Of Paris', type='Organization'), type='WORKED_AT')]


In [12]:
llm_transformer_props = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=["NATIONALITY", "LOCATED_IN", "WORKED_AT", "SPOUSE"],
    node_properties=["born_year"],
)
graph_documents_props = llm_transformer_props.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents_props[0].nodes}")
print(f"Relationships:{graph_documents_props[0].relationships}")

Nodes:[Node(id='Marie Curie', type='Person', properties={'born_year': '1867'}), Node(id='Pierre Curie', type='Person')]
Relationships:[Relationship(source=Node(id='Marie Curie', type='Person'), target=Node(id='Pierre Curie', type='Person'), type='NATIONALITY')]


In [8]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()
graph.query("""
            MATCH ()-[r]-()
            DELETE r
            """)
graph.query("""
            MATCH (n)
            DELETE n
            """)
print(graph.schema)

Node properties:

Relationship properties:

The relationships:



In [9]:
graph.add_graph_documents(
  graph_documents
)
graph.refresh_schema()
print(graph.schema)

Node properties:
Person {id: STRING}
Organization {id: STRING}
Relationship properties:

The relationships:
(:Person)-[:PROFESSOR]->(:Organization)
(:Person)-[:MARRIED]->(:Person)


In [10]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [11]:
response = chain.invoke({"query": "Who is the spouse of Marie Curie?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p1:Person)-[:MARRIED]->(p2:Person)
WHERE p1.id = "Marie Curie"
RETURN p2.id AS SpouseID;

Full Context:
[{'SpouseID': 'Pierre Curie'}]

> Finished chain.


{'query': 'Who is the spouse of Marie Curie?',
 'result': 'Pierre Curie is the spouse of Marie Curie.'}